In [ ]:
!pip install faiss-cpu langchain pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install python-telegram-bot -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community==0.2.1 langchain-core==0.2.1  sentence-transformers cohere -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import asyncio
import logging
import nest_asyncio
from telegram import Update, Document
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFaceHub, Cohere
from langchain.chains import RetrievalQA, LLMChain
from langchain_core.prompts import PromptTemplate
import langchain
langchain.debug = False

nest_asyncio.apply()

In [ ]:
hf_token = '<hf_token>'
os.environ['HF_TOKEN'] = hf_token

In [ ]:
tg_token = '<tg_token>'

In [ ]:
def setup_rag_system(docs):
    embeddings = HuggingFaceEmbeddings()
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
    llm = HuggingFaceHub(
        repo_id=repo_id,
        model_kwargs={"temperature": 0.8, "top_k": 10, "max_length":200},
        huggingfacehub_api_token=hf_token,
    )

    prompt_template = """Используй следующий контекст, чтобы ответить на вопрос.
    Если ты не знаешь ответ, просто скажи, что не знаешь, не пытайся придумать ответ.
    Выводи названия всех подходящих изделий, ориентируясь на цвет пряжи, а также ссылку на инструкцию, если есть. Выводи только подходящие варианты, цвета которых совпадают с требуемыми. Цвета изделия должны совпадать с цветами в вопросе.

    {context}

    Вопрос: {question}
    Полезный ответ:
    """
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    qa_chain = RetrievalQA.from_llm(llm,
        retriever=retriever,
        return_source_documents=True,
        prompt=PROMPT,
        llm_chain_kwargs={
            "verbose": True
        })
    return qa_chain

qa_chain = None

In [ ]:
from telegram import KeyboardButton, ReplyKeyboardMarkup

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text('Привет! У тебя осталась пряжа после вязания и ты не знаешь, что можно из нее сделать? Я подскажу идеи!!\n Для начала, отправь мне PDF файл для парсинга.')


In [ ]:
async def handle_document(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    global qa_chain

    document = update.message.document
    if document and document.file_name.lower().endswith('.pdf'):
        file = await document.get_file()
        file_path = f'./{document.file_name}'
        await file.download_to_drive(file_path)
        await update.message.reply_text(f'Файл {document.file_name} сохранён. Начинаю парсинг...')

        try:
            loader = PyPDFLoader(file_path)
            documents = loader.load()
            parsed_text = "\n\n".join([doc.page_content for doc in documents])

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
            chunks = text_splitter.split_text(parsed_text)

            await update.message.reply_text(f'Файл распаршен на {len(chunks)} чанков.')
            qa_chain = setup_rag_system(documents)
            await update.message.reply_text('Теперь отправь цвета пряжи, которые у тебя есть.')
        except Exception as e:
            await update.message.reply_text(f'Произошла ошибка при парсинге файла: {str(e)}')
    else:
        await update.message.reply_text('Пожалуйста, отправьте файл в формате PDF.')

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    global qa_chain

    if qa_chain:
        query = update.message.text
        try:
            response = qa_chain.invoke(query)
            result = response['result']
            result = result[result.find('Полезный ответ:') + len('Полезный ответ:\n'):]
            await update.message.reply_text(result)
        except Exception as e:
            await update.message.reply_text(f'Произошла ошибка при обработке запроса: {str(e)}')
    else:
        await update.message.reply_text('Пожалуйста, сначала отправьте PDF файл для парсинга.')

In [ ]:
async def main() -> None:
    application = Application.builder().token(tg_token).build()

    print('Бот запущен')

    application.add_handler(CommandHandler('start', start))
    application.add_handler(MessageHandler(filters.Document.ALL, handle_document))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    await application.run_polling()

In [ ]:
if __name__ == '__main__':
    asyncio.run(main())

Бот запущен


> Entering new LLMChain chain...
Prompt after formatting:
Используй следующий контекст, чтобы ответить на вопрос.
    Если ты не знаешь ответ, просто скажи, что не знаешь, не пытайся придумать ответ.
    Выводи названия всех подходящих изделий, ориентируясь на цвет пряжи, а также ссылку на инструкцию, если есть. Выводи только подходящие варианты, цвета которых совпадают с требуемыми. Цвета изделия должны совпадать с цветами в вопросе.

    Context:
Бежевая
пряжа
Белая
пряжа
Черная
пряжа
Ёжик
Инструкция:
https://irinely.art/patterns/ezhik-2/
Кролик
Необходимые
материалы:
Белая
пряжа
Серая
пряжа
Коричневая
пряжа
Голубая
пряжа
Кролик
Инструкция:
https://irinely.art/patterns/krolik-v-odezhde-moon-s-haaksels/
Обезьянка
Необходимые
материалы:
Белая
пряжа
Бежевая
пряжа
Коричневая
пряжа
Инструкция:
https://irinely.art/patterns/obezyanka-anfisa-nadezhda-lebedeva/

Context:
Оранжевая
пряжа
Белая
пряжа
Инструкция:
https://irinely.art/patterns/lisenok-matveeva-toys/
Совенок
Необходим